In [1]:
import tkinter as tk
from tkinter import scrolledtext, ttk
import json
import requests
from threading import Thread

class StreamDisplayApp:
    def __init__(self, root):
        self.root = root
        self.root.title("DeepSeek 流式传输演示")
        self.root.geometry("800x600")
        
        # 创建主界面组件
        self.create_widgets()
        
        # 控制变量
        self.streaming = False
        self.full_response = ""

    def create_widgets(self):
        # 顶部控制面板
        control_frame = tk.Frame(self.root)
        control_frame.pack(pady=10)
        
        tk.Label(control_frame, text="输入提示词:").pack(side=tk.LEFT)
        self.input_entry = tk.Entry(control_frame, width=50)
        self.input_entry.pack(side=tk.LEFT, padx=5)
        self.input_entry.insert(0, "请解释量子计算的基本原理")
        
        self.start_btn = tk.Button(control_frame, text="开始流式传输", command=self.start_streaming)
        self.start_btn.pack(side=tk.LEFT, padx=5)
        
        self.stop_btn = tk.Button(control_frame, text="停止", command=self.stop_streaming, state=tk.DISABLED)
        self.stop_btn.pack(side=tk.LEFT)
        
        # 显示区域
        display_frame = tk.Frame(self.root)
        display_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        tk.Label(display_frame, text="实时流式响应:").pack(anchor=tk.W)
        self.stream_display = scrolledtext.ScrolledText(
            display_frame,
            wrap=tk.WORD,
            font=('Consolas', 10),
            height=20
        )
        self.stream_display.pack(fill=tk.BOTH, expand=True)
        
        # 状态栏
        self.status_var = tk.StringVar()
        self.status_var.set("准备就绪")
        tk.Label(self.root, textvariable=self.status_var, bd=1, relief=tk.SUNKEN, anchor=tk.W).pack(fill=tk.X)

    def start_streaming(self):
        if self.streaming:
            return
            
        self.streaming = True
        self.full_response = ""
        self.stream_display.delete(1.0, tk.END)
        self.start_btn.config(state=tk.DISABLED)
        self.stop_btn.config(state=tk.NORMAL)
        self.status_var.set("正在连接DeepSeek API...")
        
        # 在新线程中启动流式请求
        Thread(target=self.fetch_stream_data, daemon=True).start()

    def stop_streaming(self):
        self.streaming = False
        self.status_var.set(f"已停止 | 共接收 {len(self.full_response)} 字符")

    def fetch_stream_data(self):
        prompt = self.input_entry.get()
        api_url = "https://api.deepseek.com/v1/chat/completions"
        headers = {
            "Authorization": "Bearer "+'sk-4f981422c9134f0796863adc193c1876',
            "Content-Type": "application/json"
        }
        data = {
            "model": "deepseek-chat",
            "messages": [{"role": "user", "content": prompt}],
            "stream": True
        }
        
        try:
            response = requests.post(
                api_url,
                headers=headers,
                json=data,
                stream=True
            )
            
            if response.status_code == 200:
                self.update_status("流式传输开始...")
                
                for line in response.iter_lines():
                    if not self.streaming:
                        break
                        
                    if line:
                        decoded_line = line.decode('utf-8')
                        
                        if decoded_line.startswith('data:'):
                            event_data = decoded_line[5:].strip()
                            
                            if event_data == "[DONE]":
                                self.update_status("流式传输完成")
                                break
                            
                            try:
                                chunk = json.loads(event_data)
                                content = chunk['choices'][0]['delta'].get('content', '')
                                
                                if content:
                                    self.full_response += content
                                    self.append_content(content)
                            except Exception as e:
                                self.append_content(f"\n[解析错误] {str(e)}\n")
                
                self.append_content("\n\n--- 完整响应 ---\n" + self.full_response)
            else:
                self.append_content(f"\n请求失败: {response.status_code}\n{response.text}")
                
        except Exception as e:
            self.append_content(f"\n发生错误: {str(e)}\n")
        finally:
            self.streaming = False
            self.root.after(0, lambda: self.start_btn.config(state=tk.NORMAL))
            self.root.after(0, lambda: self.stop_btn.config(state=tk.DISABLED))

    def append_content(self, text):
        self.root.after(0, lambda: self.stream_display.insert(tk.END, text))
        self.root.after(0, lambda: self.stream_display.see(tk.END))
        self.root.after(0, lambda: self.status_var.set(f"接收中... {len(self.full_response)} 字符"))
        self.root.update()

    def update_status(self, message):
        self.root.after(0, lambda: self.status_var.set(message))
        self.root.update()

if __name__ == "__main__":
    root = tk.Tk()
    app = StreamDisplayApp(root)
    root.mainloop()